In [1]:
import numpy as np
import joblib
import pandas as pd

In [2]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
columns = ['sex', 'length', 'diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'rings']

# Load the dataset
df = pd.read_csv(url, header=None, names=columns)
df

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [3]:
from sklearn.model_selection import train_test_split


# Split the dataset into input features and target labels
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X[X.select_dtypes(include=['float64']).columns] = X.select_dtypes(include=['float64']).astype('float32')
X['sex'] = X['sex'].map({'M': 0, 'F': 1, 'I': 2}).astype('float32')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

## Tensorflow

In [4]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

print("Available devices: %s" % tf.config.list_physical_devices())

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    strategy = tf.distribute.OneDeviceStrategy(device="/GPU:0")
else:
    strategy = tf.distribute.OneDeviceStrategy(device="/CPU:0")

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
def build_model(hidden_size=64, learning_rate=0.01):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(hidden_size, activation='relu'),
        tf.keras.layers.Dense(hidden_size/2, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate)

    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['mae'])

    return model

In [10]:
import wandb
from wandb.keras import WandbCallback

def train():
    # default hyperparameters
    config_defaults = {
        'learning_rate': 0.01,
        'hidden_size': 64,
        'epochs': 50
    }
    wandb.init(config=config_defaults)

    # Configure the model
    config = wandb.config
    with strategy.scope():
        model = build_model(hidden_size=config.hidden_size, learning_rate=config.learning_rate)

        # Train the model
        history = model.fit(
            X_train, 
            y_train, 
            validation_split=0.2, 
            epochs=config.epochs, 
            batch_size=128, 
            verbose=0, 
            callbacks=[
                WandbCallback()
            ])

    # Log the results
    metrics = {
        'val_loss': history.history['val_loss'][-1],
        'val_mae': history.history['val_mae'][-1]
    }
    wandb.log(metrics)

In [11]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_mae',
        'goal': 'minimize'
    },
    'parameters': {
        'hidden_size': {
            'values': [16, 32, 64]
        },
        'learning_rate': {
            'min': 0.001,
            'max': 0.1
        },
        'epochs': {
            'values': [25, 50, 100, 150]
        },
    }
}

sweep_id = wandb.sweep(sweep_config)

wandb.agent(sweep_id, function=train, count=50)

Create sweep with ID: jpuscrsx
Sweep URL: https://wandb.ai/xkretinin/uncategorized/sweeps/jpuscrsx


wandb: Agent Starting Run: qaupxw0y with config:
wandb: 	epochs: 150
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.022622349290021913
wandb: Currently logged in as: xkretinin. Use `wandb login --relogin` to force relogin


C:\Users\kreti\.conda\envs\condaProject\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


val_loss,▁
val_mae,▁
val_loss,4.479
val_mae,1.59221


wandb: Agent Starting Run: 5jt5m4z5 with config:
wandb: 	epochs: 50
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.030067839682365018


...

C:\Users\kreti\.conda\envs\condaProject\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


val_loss,▁
val_mae,▁
val_loss,4.22009
val_mae,1.50347


wandb: Agent Starting Run: ndxwh1kv with config:
wandb: 	epochs: 100
wandb: 	hidden_size: 16
wandb: 	learning_rate: 0.053656015811064287


C:\Users\kreti\.conda\envs\condaProject\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


val_loss,▁
val_mae,▁
val_loss,4.44993
val_mae,1.52745


## PyTorch

In [19]:
import wandb
import torch.nn as nn
import torch.optim as optim
import torch
from pprint import pprint

api = wandb.Api()
sweep = api.sweep('xkretinin/uncategorized/sweeps/jpuscrsx')

best_config = sweep.best_run().config
pprint(best_config)

wandb.init(config=best_config)
best_config = wandb.config

# default hyperparameters
config = {
    'learning_rate': 0.09599,
    'hidden_size': 32,
    'epochs': 100
}

# Configure the model
model = nn.Sequential(
    nn.Linear(8, best_config.hidden_size),
    nn.ReLU(),
    nn.Linear(best_config.hidden_size, best_config.hidden_size//2),
    nn.ReLU(),
    nn.Linear(best_config.hidden_size//2, 1)
)

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=best_config.learning_rate)
batch_size = 128

for epoch in range(best_config.epochs):
    running_mae = 0.0
    for i in range(0, len(X_train), batch_size):
        # Get a batch of data
        inputs = torch.tensor(X_train.iloc[i:i+batch_size].values, dtype=torch.float32)
        labels = torch.tensor(y_train.iloc[i:i+batch_size].values, dtype=torch.float32)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass, backward pass, and optimization
        outputs = model(inputs)
        mae = criterion(outputs, labels.view(-1, 1))
        mae.backward()
        optimizer.step()

        # Accumulate loss and MAE for this batch
        running_mae += mae

    # Print the average loss for this epoch
    avg_mae = running_mae / len(X_train)
    wandb.log({'mae': avg_mae})

# Evaluate the model on the test set
with torch.no_grad():
    inputs = torch.tensor(X_test.values, dtype=torch.float32)
    labels = torch.tensor(y_test.values, dtype=torch.float32)
    outputs = model(inputs)
    mae = criterion(outputs, labels.view(-1, 1))
    wandb.log({'test_mae': mae})

wandb: Sorting runs by +summary_metrics.val_mae


{'epochs': 100, 'hidden_size': 32, 'learning_rate': 0.09599149727636776}


mae,█▂▁▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,▁
mae,0.01234
val_mae,1.73548
